In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████| 5/5 [00:50<00:00, 10.04s/it]


In [4]:
len(deals)

50

In [6]:
deals[44].describe()

"Title: Good 'n' Fun Triple Flavor Kabobs 4-oz. Bag for $3 + free shipping w/ $35\nDetails: Get 50 cents off this best seller. Opt for store pickup (where available) tp dodge the $6.99 shipping fee or orders of $35 or more ships for free. Buy Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Good-n-Fun-Triple-Flavor-Kabobs-4-oz-Bag-for-3-free-shipping-w-35/21736382.html?iref=rss-c196"

In [11]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Refurb Apple Watch SE 40mm GPS Smartwatch for $95 + free shipping
Details: That's 68% off and the lowest price we've seen for this model in any condition. A 1-year Allstate Warranty applies. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/products/Apple/Apple-Watch-SE-40-mm-GPS-Smartwatch/489595.html?iref=rss-c142

Title: Unlocked Samsung Galaxy S25+ Androi

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [12]:
result = get_recommendations()

In [13]:
len(result.deals)

5

In [14]:
result.deals[1]

Deal(product_description='The Samsung Galaxy AI Smart Ring is a cutting-edge health and fitness companion that tracks various metrics, including workouts, sleep, and heart rate, all within a compact ring design. With a 7-day battery life, this smart ring provides continuous health monitoring and insights into your well-being. It comes bundled with a $100 Best Buy gift card, offering fantastic value for those looking to enhance their health tracking capabilities.', price=300.0, url='https://www.dealnews.com/products/Samsung/Samsung-Galaxy-AI-Smart-Ring/477024.html?iref=rss-c142')

In [15]:
from agents.scanner_agent import ScannerAgent

In [16]:
agent = ScannerAgent()
result = agent.scan()

In [17]:
result

DealSelection(deals=[Deal(product_description='This Refurbished Apple Watch SE features a lightweight design perfect for everyday use and encompasses a variety of functions catering to your fitness and health needs. The watch includes GPS capabilities, letting you track your workouts and manage notifications from your smartphone without having to carry it along. It boasts a clear display to provide you with all the necessary information at a glance. With a year-long warranty from Allstate, this smartwatch is a dependable companion for anyone looking to maintain a healthy lifestyle.', price=95.0, url='https://www.dealnews.com/products/Apple/Apple-Watch-SE-40-mm-GPS-Smartwatch/489595.html?iref=rss-c142'), Deal(product_description='The certified refurbished Sonos Era 100 is a Bluetooth Smart Speaker that delivers impressive sound quality and versatility with Alexa integration for voice control. It supports multiple connectivity options including Bluetooth and Wi-Fi, ensuring seamless stre